In [19]:
# import libraries

import cv2
import mediapipe as mp
import csv
import numpy as np
import pandas as pd
import os

In [2]:
# instantiate mp modules

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [58]:
# load and annotate img (need to do it on a single image to create the csv)
with mp_holistic.Holistic(
    static_image_mode=True) as holistic:
    
    image = cv2.imread("../../../distracted_driver_dataset/imgs/train/c0_safe/img_34.jpg")
    image_height, image_width, _ = image.shape
    
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw pose, left and right hands, and face landmarks on the image.
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#     cv2.imshow('img', annotated_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [59]:
num_coords = len(results.pose_landmarks.landmark)\
            +len(results.face_landmarks.landmark)\
            +len(results.right_hand_landmarks.landmark)\
            +len(results.left_hand_landmarks.landmark)
num_coords

543

In [60]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [61]:
with open('../data/csv/img_coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [63]:
def coord_to_csv(imgs_folder, csv_path):
    counter = 0
    dir_list = os.listdir(imgs_folder)
    for folder in dir_list:
        class_name = folder
        imgs_list = os.listdir(imgs_folder + folder)
        print(f"Annotating {class_name}")
        for img in imgs_list:
            path = imgs_folder + folder + "/" + img
            with mp_holistic.Holistic(
            static_image_mode=True) as holistic:
                image = cv2.imread(path)
                image_height, image_width, _ = image.shape
                results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                    # Extract Face landmarks
                    face = results.face_landmarks.landmark
                    face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

                    # extract left hand landmarks
                    left_hand = results.left_hand_landmarks.landmark
                    left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]\
                                                   for landmark in left_hand]).flatten())

                    # extract right hand landmarks
                    right_hand = results.left_hand_landmarks.landmark
                    right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]\
                                                   for landmark in right_hand]).flatten())
                    # Concate rows
                    row = pose_row+face_row+left_hand_row+right_hand_row

                    # Append class name 
                    row.insert(0, class_name)

                    # Export to CSV
                    with open(csv_path, mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(row)
                except:
                    pass
                counter += 1
                if counter % 100 == 0:
                    print(f'{counter} imgs annotated')

In [64]:
coord_to_csv("../../../distracted_driver_dataset/imgs/train/", "../data/csv/img_coords.csv")

Annotating c0_safe
100 imgs annotated
200 imgs annotated
300 imgs annotated
400 imgs annotated
500 imgs annotated
600 imgs annotated
700 imgs annotated
800 imgs annotated
900 imgs annotated
1000 imgs annotated
1100 imgs annotated
1200 imgs annotated
1300 imgs annotated
1400 imgs annotated
1500 imgs annotated
1600 imgs annotated
1700 imgs annotated
1800 imgs annotated
1900 imgs annotated
2000 imgs annotated
2100 imgs annotated
2200 imgs annotated
2300 imgs annotated
2400 imgs annotated
Annotating c1_texting_right
2500 imgs annotated
2600 imgs annotated
2700 imgs annotated
2800 imgs annotated
2900 imgs annotated
3000 imgs annotated
3100 imgs annotated
3200 imgs annotated
3300 imgs annotated
3400 imgs annotated
3500 imgs annotated
3600 imgs annotated
3700 imgs annotated
3800 imgs annotated
3900 imgs annotated
4000 imgs annotated
4100 imgs annotated
4200 imgs annotated
4300 imgs annotated
4400 imgs annotated
4500 imgs annotated
4600 imgs annotated
4700 imgs annotated
Annotating c2_phonecal

In [65]:
pd.read_csv("../data/csv/img_coords.csv")

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z541,v541,x542,y542,z542,v542,x543,y543,z543,v543
0,c0_safe,0.451869,0.340804,-0.414619,1.0,0.433471,0.310055,-0.396678,1.0,0.435723,...,-0.010021,0.0,0.927008,0.382200,-0.007705,0.0,0.937594,0.383414,-0.003735,0.0
1,c0_safe,0.335687,0.278907,-0.669487,1.0,0.320887,0.222015,-0.638290,1.0,0.334692,...,-0.053340,0.0,0.787920,0.217617,-0.065578,0.0,0.792619,0.214140,-0.072630,0.0
2,c0_safe,0.427172,0.244115,-0.251969,1.0,0.408947,0.205869,-0.227807,1.0,0.414374,...,0.000705,0.0,1.004587,0.404081,0.019041,0.0,0.994604,0.404329,0.023989,0.0
3,c0_safe,0.394419,0.340969,-0.333984,1.0,0.368516,0.312608,-0.330131,1.0,0.369563,...,0.012663,0.0,0.825407,0.257075,0.023968,0.0,0.818549,0.264373,0.026606,0.0
4,c0_safe,0.383105,0.315476,-0.376956,1.0,0.369031,0.277121,-0.330456,1.0,0.377492,...,0.013256,0.0,0.851849,0.603172,0.023963,0.0,0.844417,0.604855,0.029847,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3866,c6_talking,0.394669,0.411214,-0.618014,1.0,0.376108,0.362289,-0.600153,1.0,0.385334,...,-0.138222,0.0,0.979968,0.504906,-0.152757,0.0,0.992437,0.512240,-0.158848,0.0
3867,c6_talking,0.368092,0.281013,-0.483196,1.0,0.359126,0.231669,-0.458207,1.0,0.369589,...,-0.113396,0.0,0.880479,0.302281,-0.133045,0.0,0.892886,0.290581,-0.145799,0.0
3868,c6_talking,0.440365,0.338097,-0.602708,1.0,0.418802,0.310608,-0.569344,1.0,0.421868,...,0.036066,0.0,0.803858,0.531579,0.034347,0.0,0.801867,0.531422,0.035088,0.0
3869,c6_talking,0.332026,0.424834,-0.587738,1.0,0.328176,0.371459,-0.563482,1.0,0.342044,...,-0.004542,0.0,0.820223,0.206265,-0.004373,0.0,0.833012,0.205601,-0.002697,0.0
